In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
#
# Import required python packages and ThermoEngine (ENKI)
#
import numpy as np
from scipy import optimize as optim
import pandas as pd
import thermoengine as thermo
#
# Get access to a thermodynamic database (by default, the Berman (1988)/MELTS database)
#
modelDB = thermo.model.Database()
#
# Create a Python reference to the Spinel ("Mag") and Rhombehedral oxide ("Ilm") solution phase classes
#
Mag = modelDB.get_phase('SplS')
Ilm = modelDB.get_phase('Rhom')
#
# Function to validate compositional data for a phase
#
def validate_endmember_comp(moles_end, phase):
    sum = 0.0
    for i in range(0,phase.props['endmember_num']):
        sum += moles_end[i]
    if not phase.test_endmember_comp(moles_end):
        print ("Calculated composition is infeasible!")
#
# Corection terms from Ghiorso and Evans (2008) that modify the MELTS models 
# Correction terms for ulvospinel derived in Ghiorso and Evans (2008)
#
def UlvCorr(t, correctReaction=True):
    tr = 298.15
    h = - 162.0 + 284.5
    s = 0.0
    if correctReaction:
        h += 2039.106175 
        s +=    1.247790
    l1 = - 0.039452*np.sqrt(4.184)
    l2 = 7.54197e-5*np.sqrt(4.184)
    h = h + 0.5*l1*l1*(t*t-tr*tr) + (2.0/3.0)*l1*l2*(t*t*t - tr*tr*tr) + 0.25*l2*l2*(t*t*t*t - tr*tr*tr*tr)
    s = s + l1*l1*(t - tr) + l1*l2*(t*t - tr*tr) + (1.0/3.0)*l2*l2*(t*t*t - tr*tr*tr)
    return h - t*s
#
# Ghiorso and Evans (2008) used the Vinet integral; MELTS uses the Berman integral
# We must substract the latter from computed chemical potentials and add in the former.
#
def BermanVint(t, p, v0, v1, v2, v3, v4):
    pr = 1.0
    tr = 298.15
    return v0*((v1/2.0-v2)*(p*p-pr*pr)+v2*(p*p*p-pr*pr*pr)/3.0 + (1.0-v1+v2+v3*(t-tr)+v4*(t-tr)*(t-tr))*(p-pr))
def VinetVint(t, p, v0, alpha, K, Kp):
    eta = 3.0*(Kp-1.0)/2.0
    x   = 1.0
    x0  = 1.0
    pr  = 1.0
    tr  = 298.15
    
    iter = 0
    while True:
        fn = x*x*(p/10000.0) - 3.0*K*(1.0-x)*np.exp(eta*(1.0-x)) - x*x*alpha*K*(t-tr)
        dfn = 2.0*x*(p/10000.0) + 3.0*K*(1.0+eta*(1.0-x))*np.exp(eta*(1.0-x)) - 2.0*alpha*K*(t-tr)
        x = x - fn/dfn
        iter += 1
        if ((iter > 500) or (fn*fn < 1.0e-15)):
            break
    
    iter = 0
    while True:
        fn = x0*x0*(pr/10000.0) - 3.0*K*(1.0-x0)*np.exp(eta*(1.0-x0)) - x0*x0*alpha*K*(t-tr)
        dfn = 2.0*x0*(pr/10000.0) + 3.0*K*(1.0+eta*(1.0-x0))*np.exp(eta*(1.0-x0)) - 2.0*alpha*K*(t-tr)
        x0 = x0 - fn/dfn
        iter += 1
        if ((iter > 500) or (fn*fn < 1.0e-15)):
            break
    
    a  = (9.0*v0*K/(eta*eta))*(1.0 - eta*(1.0-x))*np.exp(eta*(1.0-x))
    a += v0*(t-tr)*K*alpha*(x*x*x - 1.0) - 9.0*v0*K/(eta*eta)
    a -= (9.0*v0*K/(eta*eta))*(1.0 - eta*(1.0-x0))*np.exp(eta*(1.0-x0))
    a -= v0*(t-tr)*K*alpha*(x0*x0*x0 - 1.0) - 9.0*v0*K/(eta*eta)
    
    return -a*10000.0 + p*v0*x*x*x - pr*v0*x0*x0*x0
#
# Berman integral for the reaction FeTiO3 + Fe3O4 = Fe2TiO4 + Fe2O3
#
def rBerVint(T, P):
    vIntBerMag = BermanVint(T, P, 4.452, -0.582E-6, 1.751E-12, 30.291E-6, 138.470E-10)
    vIntBerUlv = BermanVint(T, P, 4.682, 0.0, 0.0, 0.0, 0.0)
    vIntBerHem = BermanVint(T, P, 3.027, -0.479e-6, 0.304e-12, 38.310e-6, 1.650e-10)
    vIntBerIlm = BermanVint(T, P, 3.170, -0.584e-6, 1.230e-12, 27.248e-6, 29.968e-10)
    return vIntBerUlv + vIntBerHem - vIntBerMag -vIntBerIlm
#
# Vinet integral for the reaction FeTiO3 + Fe3O4 = Fe2TiO4 + Fe2O3
#
def rVinetVint(T, P):
    vIntVinetMag = VinetVint(T, P, 4.452, 30.291E-6, 171.821, 9.3387)
    vIntVinetUlv = VinetVint(T, P, 4.682, 30.291E-6, 171.821, 9.3387)
    vIntVinetHem = VinetVint(T, P, 3.027, 38.310E-6, 208.768, 1.64992)
    vIntVinetIlm = VinetVint(T, P, 3.170, 27.248E-6, 171.233, 6.21289)
    return vIntVinetUlv + vIntVinetHem - vIntVinetMag - vIntVinetIlm
#
# This method computes the free energy of the exchange reaction ...
#
def deltaG(T, P, mag_mols, ilm_mols):
    muMag = Mag.chem_potential(T, P, mol=mag_mols, endmember=2)
    muUlv = Mag.chem_potential(T, P, mol=mag_mols, endmember=4) + UlvCorr(T)
    muIlm = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=2)
    muHem = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=1)
    deltaG = muUlv + muHem - muIlm - muMag - rBerVint(T, P) + rVinetVint(T, P)
    return deltaG
#
# This next function is used by the minimizer to zero the free energy of the exchange reaction ...
#
def boundary(P, Tlims, deltaG, mag_mols, ilm_mols):
    Afun = lambda T, P=P: deltaG(T, P, mag_mols, ilm_mols)
    Tbound = optim.brentq(Afun, Tlims[0], Tlims[1])
    return Tbound
#
# Note that the properties of oxygen are defined here for consistency instead of using the built-in functions.  
# Also note that the chemical potentials of hematite and magnetite are adjusted to remove the Berman-type 
# volume integrals and replace them with the Vinet-type volume integrals to be consistent with 
# Ghiorso and Evans (2008)
#
def muO2(t, p):
    tr = 298.15
    hs = 23.10248*(t-tr) + 2.0*804.8876*(np.sqrt(t)-np.sqrt(tr)) - 1762835.0*(1.0/t-1.0/tr) \
       - 18172.91960*np.log(t/tr) + 0.5*0.002676*(t*t-tr*tr)
    ss = 205.15 + 23.10248*np.log(t/tr)  - 2.0*804.8876*(1.0/np.sqrt(t)-1.0/np.sqrt(tr)) \
       - 0.5*1762835.0*(1.0/(t*t)-1.0/(tr*tr)) + 18172.91960*(1.0/t-1.0/tr) + 0.002676*(t-tr)
    return hs - t*ss
def deltaNNO (T, P, mag_mols, ilm_mols):
    muHem  = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=1)
    muHem -= BermanVint(T, P, 3.027, -0.479e-6, 0.304e-12, 38.310e-6, 1.650e-10)
    muHem += VinetVint(T, P, 3.027, 38.310E-6, 208.768, 1.64992)
    muMag =  Mag.chem_potential(T, P, mol=mag_mols, endmember=2)
    muMag -= BermanVint(T, P, 4.452, -0.582E-6, 1.751E-12, 30.291E-6, 138.470E-10)
    muMag += VinetVint(T, P, 4.452, 30.291E-6, 171.821, 9.3387)
    muOxy  = muO2(T, P)
    logfO2 = (6.0*muHem - 4.0*muMag -  muOxy)/(8.3144598*T)/np.log(10.0)
    return logfO2 - (-25018.7/T + 12.981 + 0.046*(P-1.0)/T -0.5117*np.log(T))
#
# The method below is used by the minimizer to evaluate the free energy change of the Fe-Mg exchange reaction ...
#
def deltaGfemg(T, P, mag_mols, ilm_mols):
    muSpn  = Mag.chem_potential(T, P, mol=mag_mols, endmember=3)
    muSpn -= BermanVint(T, P, 3.977, -0.489E-6, 0.0, 21.691E-6, 50.528E-10)
    muSpn += VinetVint(T, P, 3.977, 21.691E-6, 204.499, 4.0)
    
    muHer  = Mag.chem_potential(T, P, mol=mag_mols, endmember=1)
    muHer -= BermanVint(T, P, 0.973948*4.184, 0.0, 0.0, 0.0, 0.0)
    muHer += VinetVint(T, P, 0.973948*4.184, 21.691E-6, 204.499, 4.0)
    
    muIlm  = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=2)
    muIlm -= BermanVint(T, P, 3.170, -0.584e-6, 1.230e-12, 27.248e-6, 29.968e-10)
    muIlm += VinetVint(T, P, 3.170, 27.248E-6, 171.233, 6.21289)
    
    muGei  = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=0)
    muGei -= BermanVint(T, P, 3.086, -0.584e-6, 1.230e-12, 27.248e-6, 29.968e-10)
    muGei += VinetVint(T, P, 3.086, 27.2476341e-6, 171.240, 6.21527)
    
    deltaG = muSpn + muIlm - muHer - muGei
    return deltaG
#
# Calculate the activity of TiO2 relative to rutile saturation 
#
Rut = modelDB.get_phase('Rt')
def aTiO2(T, P, mag_mols, ilm_mols):
    muUlv  = Mag.chem_potential(T, P, mol=mag_mols, endmember=4) + UlvCorr(T, correctReaction=False)
    muUlv -= BermanVint(T, P, 4.682, 0.0, 0.0, 0.0, 0.0)
    muUlv += VinetVint(T, P, 4.682, 30.291E-6, 171.821, 9.3387)
    muIlm  = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=2)
    muIlm -= BermanVint(T, P, 3.170, -0.584e-6, 1.230e-12, 27.248e-6, 29.968e-10)
    muIlm += VinetVint(T, P, 3.170, 27.248E-6, 171.233, 6.21289)
    muRut = Rut.chem_potential(T, P)
    return np.exp(-(muRut+muUlv-2.0*muIlm)/(8.3143*T))

#  Fe-Ti oxide geothermobarometer
This app implements the Fe-Ti oxide geothermometer and oxygen barometer of  

Ghiorso MS and Evans BW (2008)  
*Thermodynamics of Rhombohedral Oxide Solid Solutions and a Revision of the Fe-Ti Two-oxide Geothermometer and Oxygen-barometer*  
American Journal of Science **308**, 957-1039  

and is built using the [ENKI ThermoEngine thermodynamic modeling package](https://gitlab.com/ENKI-portal/ThermoEngine). Temperatures, oxygen fugacities, and melt titania activity are calculated from compositional data on coesiting Fe-Ti oxides (i.e., magnetite and ilmenite).  Input may be entered on the interface or supplied as an Excel workbook. Processing the later will generate an Excel Workbook of calculation results. Source code may be downloaded from the[the app's GitLab repository](https://gitlab.com/ENKI-portal/app-fe-ti-oxide-geotherm).    

Calculated results are:
- **Fe-Ti exchange temperature** from the equilibrium: FeTiO<sub>3</sub> (ilm) + Fe<sub>3</sub>O<sub>4</sub> (mag) = Fe<sub>2</sub>TiO<sub>4</sub> (mag) + Fe2O<sub>3</sub> (ilm)
- **Log<sub>10</sub> oxygen fugacity relative to the nickel-nickel oxide buffer** from the equilibrium: O<sub>2</sub> + 4 Fe<sub>3</sub>O<sub>4</sub> (mag) = 6 Fe<sub>2</sub>O<sub>3</sub> (ilm)
- **Fe-Mg exchange temperature** from the equilibrium: FeAl<sub>2</sub>O<sub>4</sub> (mag) + MgTiO<sub>3</sub> (ilm) = MgAl<sub>2</sub>O<sub>4</sub> (mag) + FeTiO<sub>3</sub> (ilm)
- The **activity of TiO<sub>2</sub> relative to rutile saturation** in the melt coexisting with the two oxides using the method of Ghiorso and Gualda (2012, *Contributions to Mineralogy and Petrology*, **165[1]**, DOI: 10.1007/s00410-012-0792-y)

*Note: Fe may be input as FeO or Fe<sub>2</sub>O<sub>3</sub> as the input values are adjusted. The cation-anion ratio of the phase is used to compute the ferric/ferrous ratio to maintain charge balance and phase stoichiometry.*
### Enter compositions, or download an Excel template, or upload an Excel workbook; Calculate for results 

In [34]:
from __future__ import division
import ipywidgets as ipw

def create_expanded_button(description, button_style):
    return ipw.Button(description=description, button_style=button_style, 
                      layout=ipw.Layout(height='auto', width='auto'))
        
def mk_btn(description):
    btn = ipw.Button(description=description, layout=ipw.Layout(width="45px"))
    btn.on_click(on_click)
    return btn

def mk_ft(description, value, disable=False):
    txt = ipw.FloatText(description=description, value=value, disabled=disable,
                       layout=ipw.Layout(width="170px"))
    return txt

def mk_HTML(description):
    html = ipw.HTML(value=description, layout=ipw.Layout(width="200px"))
    return html

txt_entries = [
    mk_ft('Mag SiO2', 0.0),   mk_ft('Ilm SiO2', 0.0),   
    mk_ft('Mag TiO2', 4.35),  mk_ft('Ilm TiO2', 28.73), 
    mk_ft('Mag Al2O3', 1.94), mk_ft('Ilm Al2O3', 0.35), 
    mk_ft('Mag Fe2O3', 0.0),  mk_ft('Ilm Fe2O3', 0.0),  
    mk_ft('Mag Cr2O3', 0.18), mk_ft('Ilm Cr2O3', 0.0),  
    mk_ft('Mag FeO', 86.34),  mk_ft('Ilm FeO', 65.98),
    mk_ft('Mag MnO', 0.44),   mk_ft('Ilm MnO', 0.23),
    mk_ft('Mag MgO', 1.2),    mk_ft('Ilm MgO', 1.02),
    mk_ft('Mag CaO', 0.0),    mk_ft('Ilm CaO', 0.0),
    mk_ft('Mag Na2O', 0.0),   mk_ft('Ilm Na2O', 0.0),
]

upload = ipw.FileUpload(description='Upload Excel file', 
                        accept='.xlsx', 
                        multiple=False,
                        layout=ipw.Layout(width="170px"),
                        button_style='success')

progress = ipw.IntProgress(value=0, min=0, max=1, description='Progress:',
                          bar_style='success', orientation='horizontal')

case_lbl = ipw.Label(value='Processing: interface composition',
                    layout=ipw.Layout(width="300px"))

download = ipw.HTML(value="<a href='./GE08_test.xlsx' download='input.xlsx'>Excel input template",
                    placeholder='Download',
                    description='Download',
                    layout=ipw.Layout(width="300px"))

txt_results = [
    mk_ft('P (bars)', 2000.0),
    mk_ft('T °C FeTi', 0),
    mk_ft('dNNO', 0),
    mk_ft('T °C FeMg', 0),
    mk_ft('aTiO2', 0),
    mk_HTML(''),
    upload,
    progress,
    case_lbl,
    download
]

def on_click(btn):
    global case_lbl, progress, upload, download
    keys = ['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'Cr2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O']
    batch_mode = False
    progress.value = 0
    if bool(upload.value):
        # ['SiO2', 'TiO2', 'Al2O3', 'V2O3', 'Cr2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'ZnO', 'NiO']
        data_frame = pd.read_excel(upload.data[0], header=1)
        data = np.array(data_frame)
        (row, col) = data.shape
        OX = np.hstack((
            data[:, 2:5],  np.zeros((row,1)), data[:, 6:11], np.zeros((row,1)),
            data[:,13:16], np.zeros((row,1)), data[:,17:22], np.zeros((row,1))
        ))
        batch_mode = True
        progress.max = row
        T_FeTi = []
        T_FeMg = []
        delta_NNO = []
        act_TiO2 = []
    else:
        entry  = [txt_entries[2*i].value for i in range(0,10)]
        entry += [txt_entries[2*i+1].value for i in range(0,10)]
        OX = np.array([entry])
        row = 1
    for i in range(0,row):
        if batch_mode:
            case_lbl.value = 'Processing: ' + data[i,1]
        sp_wt = dict(zip(keys,list(OX[i][0:10])))
        rh_wt = dict(zip(keys,list(OX[i][10:20])))
        Mag_mol_oxides = thermo.chem.format_mol_oxide_comp(sp_wt, convert_grams_to_moles=True)
        Ilm_mol_oxides = thermo.chem.format_mol_oxide_comp(rh_wt, convert_grams_to_moles=True)
        #
        Mag_moles_end = Mag.calc_endmember_comp(
            mol_oxide_comp=Mag_mol_oxides, method='intrinsic', normalize=True)
        validate_endmember_comp(Mag_moles_end, Mag)
        Ilm_moles_end = Ilm.calc_endmember_comp(
            mol_oxide_comp=Ilm_mol_oxides, method='intrinsic', normalize=True)
        validate_endmember_comp(Ilm_moles_end, Ilm)
        #
        P = txt_results[0].value
        Teq = boundary(P, [500.,2000.], deltaG, Mag_moles_end, Ilm_moles_end)
        if batch_mode:
            T_FeTi.append(Teq-273.15)
        else:
            txt_results[1].value = round(Teq-273.15,2)
        #
        logNNO = deltaNNO(Teq, P, Mag_moles_end, Ilm_moles_end)
        if batch_mode:
            delta_NNO.append(logNNO)
        else:
            txt_results[2].value = round(logNNO,3)
        #
        Tlow  = deltaGfemg(500.0, P, Mag_moles_end, Ilm_moles_end)
        Thigh = deltaGfemg(2000.0, P, Mag_moles_end, Ilm_moles_end)
        if np.sign(Tlow) != np.sign(Thigh):
            Tfemg = boundary(P, [500.,2000.], deltaGfemg, Mag_moles_end, Ilm_moles_end)
            if batch_mode:
                T_FeMg.append(Tfemg-273.15)
            else:
                txt_results[3].value = round(Tfemg-273.15,2)
        else:
            if batch_mode:
                T_FeMg.append('None')
            else:
                txt_results[3].value = 0.0
                txt_results[5].value = "<b><font color='red'>No Fe-Mg temperature found.</b>"
        #
        activ = aTiO2(Teq, P, Mag_moles_end, Ilm_moles_end)
        if batch_mode:
            act_TiO2.append(activ)
        else:
            txt_results[4].value = round(activ, 4)
        progress.value = i
    if batch_mode:
        data_frame.columns = [
            'Index', 'Label', 
            'Spinel SiO2', 'Spinel TiO2', 'Spinel Al2O3', 'Spinel V2O3','Spinel Cr2O3', 
            'Spinel FeO', 'Spinel MnO', 'Spinel MgO', 'Spinel CaO', 'Spinel ZnO', 'Spinel NiO',
            'Rhomb SiO2', 'Rhomb TiO2', 'Rhomb Al2O3', 'Rhomb V2O3', 'Rhomb Cr2O3', 'Rhomb FeO', 
            'Rhomb MnO', 'Rhomb MgO', 'Rhomb CaO', 'Rhomb ZnO', 'Rhomb NiO']
        data_frame['fO2 rel NNO'] = np.array(delta_NNO)
        data_frame['T °C, Fe-Ti'] = np.array(T_FeTi)
        data_frame['T °C, Fe-Mg'] = np.array(T_FeMg)
        data_frame['TiO2 activity'] = np.array(act_TiO2)
        case_lbl.value = 'File processing completed'
        progress.value = 0
        data_frame.to_excel('output.xlsx')
        download.value = value="<a href='./output.xlsx' download='output.xlsx'>Excel file results"
    
grid1 = ipw.GridBox(txt_entries, layout=ipw.Layout(grid_template_columns="repeat(2,200px)"))
grid1.box_style = "info"
grid2 = ipw.GridBox(txt_results, layout=ipw.Layout(grid_template_columns="repeat(1,100px)"))
grid2.box_style = "warning"

title = create_expanded_button('Enter oxide compositions in wt%', 'info')
compute = create_expanded_button('Calculate', 'success')
compute.on_click(on_click)
left = create_expanded_button('input', 'warning')

ipw.AppLayout(header=title,
              left_sidebar=left,
              center=grid1,
              right_sidebar=grid2,
              footer=compute,
              pane_widths=[1,3,2],
              pane_heights=[1,12,1]
             )

AppLayout(children=(Button(button_style='info', description='Enter oxide compositions in wt%', layout=Layout(g…